In [11]:
import json
import sys
sys.path.append('../src')
from pydantic import BaseModel

# Define the Pydantic model
class ExperimentConfig(BaseModel):
    epochs: int
    learning_rate: float
    batch_size: int

# Path to your JSON file
json_file_path = '/home/xmootoo/Documents/VSCode/ssl-seizure-detection/exp/1.0/test.json'

# Load and parse the JSON file
with open(json_file_path, 'r') as file:
    data = json.load(file)

# Assuming you want to load the configuration for 'experiment_1'
experiment_data = data['experiment_1']

# Create an instance of the ExperimentConfig with the loaded data
config = ExperimentConfig(**experiment_data)

# config now contains the configuration from the JSON file
print(config)

epochs=10 learning_rate=0.001 batch_size=32
